In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
# devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
# testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [4]:
print(testSentences[3])

[['Die', 'O'], ['These', 'O'], [',', 'O'], ['Schlatter', 'B-PER'], ['sei', 'O'], ['Antisemit', 'O'], ['gewesen', 'O'], [',', 'O'], ['wurde', 'O'], ['seither', 'O'], ['in', 'O'], ['der', 'O'], ['theologischen', 'O'], ['Fachliteratur', 'O'], ['nicht', 'O'], ['mehr', 'O'], ['vertreten', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [6]:
print(len(labelSet))
print(models.max_sequence_length)

25
56


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'O': 21, 'B-PERderiv': 1, 'I-ORG': 2, 'I-PERpart': 3, 'I-PERderiv': 4, 'B-LOCderiv': 17, 'I-OTHderiv': 18, 'I-LOCpart': 5, 'B-OTHderiv': 6, 'I-LOCderiv': 7, 'B-PERpart': 20, 'B-ORGpart': 19, 'I-ORGpart': 9, 'PADDING_TOKEN': 0, 'B-PER': 8, 'B-LOC': 10, 'B-LOCpart': 11, 'B-ORG': 12, 'I-ORGderiv': 22, 'I-PER': 13, 'I-OTH': 23, 'B-OTHpart': 14, 'I-LOC': 24, 'B-ORGderiv': 25, 'I-OTHpart': 15, 'B-OTH': 16}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'PADDING_TOKEN': 0, 'initialUpper': 4, 'allLower': 2, 'numeric': 1, 'allUpper': 3, 'other': 5, 'contains_digit': 7, 'mainly_numeric': 6}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'S': 5, 'è': 168, 'ă': 6, 'а': 114, '鷹': 8, 'Œ': 248, 'ب': 270, 'أ': 11, 'ラ': 12, 'C': 13, '李': 7, 'ū': 15, '£': 207, '\xad': 16, '†': 17, 'j': 18, 'ą': 19, 'å': 20, 't': 21, 'ã': 330, '‹': 22, 'ي': 24, '°': 198, '@': 25, 'έ': 121, 'φ': 274, '?': 27, 'u': 28, 'ċ': 35, 'ǒ': 33, 'ņ': 291, 'υ': 32, 'ě': 60, '術': 34, '±': 30, '&': 36, '九': 38, 'ж': 39, '樓': 40, 'ệ': 41, '⋅': 232, 'Ä': 42, '–': 293, '»': 219, 'κ': 234, '妃': 43, 'a': 44, 'k': 46, 'Š': 48, 'İ': 49, '▪': 14, 'о': 50, 'È': 237, '3': 51, 'F': 52, 'Т': 54, '2': 83, 'ä': 55, 'µ': 56, 'м': 57, 'ń': 58, 'ε': 59, 'q': 61, 'r': 308, 'y': 63, 'ế': 65, 'x': 240, '(': 66, '\x96': 67, 'ó': 68, 'V': 69, '¹': 269, 'ض': 70, '‘': 72, 'η': 264, 'ý': 73, 'ь': 244, 'ō': 74, '별': 126, 'à': 81, 'г': 79, 'A': 80, 'у': 82, '´': 84, '寝': 85, ',': 107, 'ā': 86, '€': 87, 'Ø': 89, 'ř': 243, 'к': 90, '−': 299, 'В': 91, 'И': 93, 'и': 285, '佐': 94, 'á': 95, 'л': 187, '·': 96, "'": 97, 'ḫ': 250, 'T': 100, '\x94': 105, 'ο': 301, 'ž': 103, 'ñ': 252, '!': 328

In [14]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

26
26


# Test Model

In [ ]:
# import importlib
# importlib.reload(models)

In [ ]:
tmp_model_filename = 'model_lstm_germeval.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_lstm_v2()

In [ ]:
#import importlib
#importlib.reload(utils)
# print(models.max_sequence_length)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm_germeval.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'model_lstm_germeval.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_lstm_v2()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=512), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case

Epoch 1/10
1499/1500 [============================>.] - ETA: 0s - loss: 0.0446 - acc: 0.9838
New maximum F1 score: 0.7637277648878575 (before: 0) Saving to model_lstm_germeval.1.h5
1500/1500 [==============================] - 579s 386ms/step - loss: 0.0446 - acc: 0.9838 - val_loss: 0.0012 - val_acc: 0.9891
Epoch 2/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0203 - acc: 0.9890
New maximum F1 score: 0.7850779510022272 (before: 0.7637277648878575) Saving to model_lstm_germeval.1.h5
1500/1500 [==============================] - 569s 379ms/step - loss: -0.0203 - acc: 0.9890 - val_loss: -0.0473 - val_acc: 0.9894
Epoch 3/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0740 - acc: 0.9901
New maximum F1 score: 0.7954964931709116 (before: 0.7850779510022272) Saving to model_lstm_germeval.1.h5
1500/1500 [==============================] - 575s 383ms/step - loss: -0.0740 - acc: 0.9901 - val_loss: -0.1003 - val_acc: 0.9896
Epoch 4/10
1499/1500 [==============

1499/1500 [============================>.] - ETA: 0s - loss: -0.1844 - acc: 0.9914
New maximum F1 score: 0.8160962767957879 (before: 0.808390022675737) Saving to model_lstm_germeval.2.h5
1500/1500 [==============================] - 572s 381ms/step - loss: -0.1844 - acc: 0.9914 - val_loss: -0.2101 - val_acc: 0.9911
Epoch 6/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.2395 - acc: 0.9920
New maximum F1 score: 0.8195684523809526 (before: 0.8160962767957879) Saving to model_lstm_germeval.2.h5
1500/1500 [==============================] - 571s 381ms/step - loss: -0.2395 - acc: 0.9920 - val_loss: -0.2636 - val_acc: 0.9915
Epoch 7/10
1500/1500 [==============================] - 575s 383ms/step - loss: -0.2943 - acc: 0.9926 - val_loss: -0.3160 - val_acc: 0.9908
Epoch 8/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.3488 - acc: 0.9929
New maximum F1 score: 0.820959081651546 (before: 0.8195684523809526) Saving to model_lstm_germeval.2.h5
1500/1500 [=======

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close